In [36]:
from openai import OpenAI
import os
import io
import base64
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_api_key)


In [37]:
from PIL import Image

salmon_image = Image.open("images/salmon.jpg")
print(f"size: {salmon_image.size}")
print(f"mode: {salmon_image.mode}")
print(f"format: {salmon_image.format}")



size: (1500, 1125)
mode: RGB
format: JPEG


In [61]:

def encode_image_to_base64(image_input, format="JPEG"):
    """
    Encodes an image to a Base64 string.

    :param image_input: Either a file path (str) or a PIL.Image.Image instance
    :param format: Format to use when encoding a PIL image (default: 'PNG')
    :return: Base64 encoded string
    """
    if isinstance(image_input, str):
        # It's a file path
        with open(image_input, "rb") as f:
            encoded_string = base64.b64encode(f.read()).decode("utf-8")
    elif isinstance(image_input, Image.Image):
        # It's a PIL image
        buffered = io.BytesIO()
        image_format = image_input.format or format
        image_input.save(buffered, format=image_format)
        encoded_string = base64.b64encode(buffered.getvalue()).decode("utf-8")
    else:
        raise TypeError("image_input must be a file path (str) or a PIL.Image.Image instance.")
    return encoded_string

In [62]:
def query_openai_vision(client, image, prompt, model="gpt-4o", max_tokens=100):
    base_64_image= encode_image_to_base64(image)
    try:
        messages = [{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base_64_image}"
                    }
                }
            ]
        }]
    except Exception:
        return "Error"
    


    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens
    )

    return response.choices[0].message.content

In [63]:
food_prompt = """
Context: You are a world-class nutritionist and food analysis AI. Your expertise is in visually recognizing foods from images and estimating their nutritional information, especially calorie content.
Instruction: Given an image of a dish, carefully analyze the visual features
Input:[the image i am about to share]
Output: Provide the name of the food, a brief description of what you see and nutritional profile.
"""
print(food_prompt)


Context: You are a world-class nutritionist and food analysis AI. Your expertise is in visually recognizing foods from images and estimating their nutritional information, especially calorie content.
Instruction: Given an image of a dish, carefully analyze the visual features
Input:[the image i am about to share]
Output: Provide the name of the food, a brief description of what you see and nutritional profile.



In [65]:
query = query_openai_vision(
    openai_client,
    salmon_image,
    food_prompt
)
print(query)

**Food Name:** Grilled Salmon with Green Beans

**Description:** 
The dish features a piece of grilled salmon, seasoned with herbs and spices, giving it a crispy exterior. Accompanied by a side of fresh green beans, likely steamed or sautéed, and a lemon wedge for added flavor.

**Nutritional Profile:**

- **Grilled Salmon (Approx. 6 oz):**
  - Calories: ~367
  - Protein: ~39g
  - Fat:


In [66]:
promt_2 = food_prompt = """
Context: 
You are a world-class nutritionist and food analysis AI. 
Your expertise is in visually recognizing foods from images and estimating their nutritional information, especially calorie content.

Instruction:
Given an image of a dish, carefully analyze the visual features to:
- Identify the primary food item(s) in the image.
- Estimate the approximate total calories of the dish.
- Briefly list the likely main ingredients.
- Classify the dish as breakfast, lunch, dinner, or snack if possible.

If you are uncertain about the dish, mark fields as "unknown".

Return your answer in the following JSON format:

{
  "dish_name": "",
  "estimated_calories": "",
  "main_ingredients": [],
  "meal_type": ""
}

Input:
<Food Image>

Output:
A structured JSON object as described above.
"""

query_2 = query_openai_vision(
    openai_client,
    salmon_image,
    promt_2
)
print(query_2)

```json
{
  "dish_name": "Salmon with Green Beans",
  "estimated_calories": "450",
  "main_ingredients": ["salmon", "green beans", "lemon", "seasonings"],
  "meal_type": "dinner"
}
```
